In [ ]:
import requests

class PaymentGatewayError(Exception):
    """Base class for all payment errors"""
    pass

class PaymentDeclined(PaymentGatewayError):
    """Raised when bank declines transaction"""
    pass

class GatewayTimeout(PaymentGatewayError):
    """Raised when the API does not respond"""
    pass

def charge_user(amount):
    try:
        response = requests.post("https://api.payment.com/charge", json={"amt": amount}, timeout=5)
        response.raise_for_status()

    except requests.exceptions.Timeout as e:
        # Chain the exception to preserve the root cause but present a clean API
        raise GatewayTimeout("The payment provider is down.") from e

    except requests.exceptions.HTTPError as e:
        if response.status_code == 402:
            raise PaymentDeclined("Card was declined.") from e
        raise PaymentGatewayError(f"Unexpected error: {response.status_code}") from e

# Now the calling code only handles *your* exceptions, not the library's.
try:
    charge_user(100)
except PaymentGatewayError as e:
    # Handle all payment issues here
    print(f"Payment failed: {e}")